In [ ]:
import torch
import transformers
from torch.utils.data import DataLoader, Dataset
from transformers import GPT2Tokenizer, GPT2Model, GPT2LMHeadModel

MAX_LEN = 256 
BATCH_SIZE = 32 
LR = 1e-4 
EPOCHS = 10 
P_DROP = 0.1 
P_BLANK = 0.1 

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Define the missing functions
class TextDataset(Dataset):
    def __init__(self, texts):
        self.texts = texts

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        encoding = tokenizer.encode_plus(text, truncation=True, max_length=MAX_LEN, padding='max_length')
        return encoding['input_ids'], encoding['attention_mask'], encoding['input_ids']

def get_corpus_loader(batch_size, max_len):
    corpus_texts = []  # Load your corpus texts here
    dataset = TextDataset(corpus_texts)
    return DataLoader(dataset, batch_size=batch_size)

def get_author_loader(author, batch_size, max_len):
    author_texts = []  # Load your author-specific texts here
    dataset = TextDataset(author_texts)
    return DataLoader(dataset, batch_size=batch_size)

def create_noisy_input(input_ids, p_drop, p_blank):
    mask = torch.rand(input_ids.shape) < p_drop
    noisy_input_ids = input_ids.masked_fill(mask, tokenizer.pad_token_id)
    mask = torch.rand(input_ids.shape) < p_blank
    noisy_input_ids = noisy_input_ids.masked_fill(mask, tokenizer.mask_token_id)
    return noisy_input_ids

def get_input_text(text):
    encoding = tokenizer.encode_plus(text, truncation=True, max_length=MAX_LEN, padding='max_length')
    return torch.tensor(encoding['input_ids']).unsqueeze(0), torch.tensor(encoding['attention_mask']).unsqueeze(0)


In [ ]:
corpus_loader = get_corpus_loader(BATCH_SIZE, MAX_LEN)
model.train() 
optimizer = torch.optim.Adam(model.parameters(), lr=LR) 
for epoch in range(EPOCHS):
  for batch in corpus_loader:
    input_ids, attention_mask, labels = batch 
    outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels) 
    loss = outputs.loss 
    loss.backward()
    optimizer.step()
    optimizer.zero_grad() 

authors = ['Sir Arthur Conan Doyle', 'Charles Dickens', 'George Alfred Henty', ...] 
for author in authors:
  author_loader = get_author_loader(author, BATCH_SIZE, MAX_LEN) 
  model.train() 
  optimizer = torch.optim.Adam(model.parameters(), lr=LR) 
  for epoch in range(EPOCHS):
    for batch in author_loader:
      input_ids, attention_mask, labels = batch 
      noisy_input_ids = create_noisy_input(input_ids, P_DROP, P_BLANK)
      outputs = model(input_ids=noisy_input_ids, attention_mask=attention_mask, labels=input_ids) 
      loss = outputs.loss
      loss.backward() 
      optimizer.step()
      optimizer.zero_grad() 

text1 = "The front desk staff was professional and very helpful." 
text2 = "The staff at reception were courteous and helpful."
input_ids1, attention_mask1 = get_input_text(text1) 
input_ids2, attention_mask2 = get_input_text(text2)

model.eval() 


with torch.no_grad(): 
  outputs1 = model(input_ids=input_ids1, attention_mask=attention_mask1, labels=input_ids1) 
  loss1 = outputs1.loss  
  perplexity1 = torch.exp(loss1) 


with torch.no_grad(): 
  outputs2 = model(input_ids=input_ids2, attention_mask=attention_mask2, labels=input_ids2) 
  loss2 = outputs2.loss 
  perplexity2 = torch.exp(loss2) 


if abs(perplexity1 - perplexity2) < 0.1: 
  print("The texts have similar style.")
else:
  print("The texts have different style.")
